In [2]:
# HIDDEN
import pandas as pd

## Multiclass Classification (outline)

Should this be a separate section or just an addendum to an existing section?

- What is multiclass classification
    - More than 2 labels that can be classified; versus a binary classification which is restricted to 2 labels
- A situation where multiclass classification might be used
    - Preferably come up with an example in lebron.csv, but might have to use a different data set
- OvO vs OvR
    - In logistic regression, multiclass classification comes down to a bunch of binary problems
    - OvO: Fit a binary classification problem to each pair of classes, then vote for class that wins the most "head-to-head" classifications
    - OvR/OvA: Fit a binary classification problem for each class versus the rest of the classes; result is class that has the highest probability
- Example walking through multiclass? sklearn's logistic regression supports OvR out of the box.

## Other types of classification problems

- Multilabel: More than one classification problem (e.g. a document can be positive/negative, religion/not religion, political/apolitical, etc.)

## Multiclass Classification

So far we have discussed binary classification, in which our classifier determines whether an observation is part of the positive class or the negative class. However, many data science problems involve **multiclass classification**, in which we would like to classify observations as one of several different classes. For example, we may be interested in knowing whether an animal is a dog, a cat, a squirrel, or a raccoon. In practice, we use **one-vs-rest (OvR) classification** to solve these types of problems.

### One-Vs-Rest Classification

In OvR classification (also known as one-vs-all, or OvA), we decompose a multiclass classification problem into several different binary classification problems. Returning to the example of animal identification, each observation $X_i$ in the training data would be assigned a label $y_i$ that classifies the animal as either a dog, a cat, a squirrel, or a raccoon. Then for each unique label $k$, we construct a new label vector $z$ where $z=1$ if $y=k$ and $z=0$ if $y \neq k$. With our four new label vectors, we can then create four separate classification tasks that predicts whether an observation belongs to the positive class. 

So far in binary classification, we have used the output of these predictors - for example, we were primarily interested in whether the observation was a dog or not a dog. However, in multiclass classification we use the probabilities that each classifier outputs, then we assign the label $k$ for which the respective classifier outputs the highest probability score. Suppose the four classifiers output the probability scores shown below:

In [10]:
# HIDDEN
probas = pd.DataFrame([[0.75, 0.25], [0.63, 0.37], [0.18, 0.82], [0.44, 0.56]], 
                      index=['Dog', 'Cat', 'Squirrel', 'Raccoon'],
                      columns=['P(y=k)', 'P(y!=k)'])

In [11]:
probas

,P(y=k),P(y!=k)
Dog,0.75,0.25
Cat,0.63,0.37
Squirrel,0.18,0.82
Raccoon,0.44,0.56


Since the Dog classifier outputs the highest probability score, we predict that our observation is a dog.

### Iris Dataset

In [12]:
from sklearn.datasets import load_iris

iris = load_iris()

In [14]:
print(iris.DESCR)

Iris Plants Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20  0.76     0.9565  (high!)
    ============== ==== ==== ======= ===== ====================

    :Missing Attribute Values: None
  

## Multilabel Classification

Another type of classification problem is **multilabel classification**, in which our goal is to solve more than one classification problem for the same dataset. An example would be a document classification system: a document can be classified as having positive or negative sentiments, but it can also be distinguished between religious/nonreligious or political/apolitical. Multilabel problems can also be multiclass; we may want our document classification system to distinguish between a list of genres, or identify the language that the document is written in. 

Because each set of labels in a multiclass classification is mutually exclusive, each problem can be solved independently of the others. We can then combine the outputs so that every observation is assigned a set of classification labels.

## Summary

(to-do)